In [2]:
import ast

import numpy as np
import pandas as pd

from pysgpp.extensions.datadriven.learner import LearnerBuilder
from pysgpp.extensions.datadriven.uq.plot.plot1d import plotSG1d
from pysgpp.extensions.datadriven.uq.plot.plot2d import plotSG2d

import matplotlib.pyplot as plt

In [4]:
filename = 'output_100_configs.csv'

df = pd.read_csv(filename, header = None, names = ['params', 'time'])
df.params = df.params.apply(ast.literal_eval)

for param in df.iloc[0].params.keys():
    df[param] = df.params.apply(lambda row: row[param])
    
df['density'] = (df.MOL_X*df.MOL_Y)/(df.DOMAIN_SIZE_X*df.DOMAIN_SIZE_Y)
del df['params']
df = df[df.time > 0]

In [10]:
df[df.BLOCK_SIZE>1]

,time,DOMAIN_SIZE_X,TEMP,DOMAIN_SIZE_Y,DELTA_T,V_x,V_y,LINKED_CELL_SIZE_Y,LINKED_CELL_SIZE_X,DOMAIN_SIZE,MOL_X,MOL_Y,CUTOFF_RADIUS,TIMESTEPS,BLOCK_SIZE,CHECKPOINT,density
2,77.665511,40,2.5,40,0.002,1,1.5,26,21,5,42,41,3.75,1000,10,checkpoints/TX65RKKAM3_999,1.07625
6,113.095191,40,2.5,40,0.002,1,1.5,11,36,5,42,41,3.25,1000,10,checkpoints/PYQRTFSES9_999,1.07625
38,46.908025,40,2.5,40,0.002,1,1.5,16,26,5,44,41,2.75,1000,10,checkpoints/GYG7VNJN70_999,1.12750
50,44.128947,40,2.5,40,0.002,1,1.5,21,16,5,42,41,4.50,1000,10,checkpoints/TW2AOXSHBC_999,1.07625


In [6]:
params = ['DOMAIN_SIZE_X', 'DOMAIN_SIZE_Y', 'MOL_X', 'MOL_Y', 'LINKED_CELL_SIZE_X', 'LINKED_CELL_SIZE_Y',
          'BLOCK_SIZE', 'CUTOFF_RADIUS', 'density']

In [7]:
num_samples = df.shape[0]

X = df[params].values
Y = df.time.values

In [8]:
levels = 5 
num_dims = len(params)

builder = LearnerBuilder()
builder.buildRegressor()
builder.withTrainingDataFromNumPyArray(X, Y)
builder = builder.withGrid()
builder.withLevel(levels)
builder = builder.withSpecification()
builder.withLambda(0.0001)

builder.withIdentityOperator()
builder = builder.withStopPolicy()
builder = builder.withCGSolver()
builder.withAccuracy(0.000100)
builder.withImax(500)
learner = builder.andGetResult()

learner.learnData()

<pysgpp.pysgpp_swig.DataVector; proxy of <Swig Object of type 'sgpp::base::DataVector *' at 0x7f277a64d150> >

In [15]:
learner.alpha.array()

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [16]:
X

array([[ 59.        ,  59.        ,  74.        ,  61.        ,
         36.25      ,  38.5       ,  14.75      ,  17.75      ,
          1.29675381],
       [ 59.        ,  59.        ,  67.        ,  53.        ,
         36.75      ,  35.        ,  14.75      ,  16.75      ,
          1.02010916],
       [ 59.        ,  59.        ,  67.        ,  46.        ,
         38.75      ,  39.5       ,  14.75      ,  17.        ,
          0.88537777],
       [ 59.        ,  59.        ,  71.        ,  70.        ,
         38.25      ,  39.75      ,  14.75      ,  17.25      ,
          1.42775065],
       [ 59.        ,  59.        ,  47.        ,  69.        ,
         38.        ,  36.75      ,  14.75      ,  16.75      ,
          0.93162884],
       [ 59.        ,  59.        ,  56.        ,  46.        ,
         37.75      ,  39.        ,  14.75      ,  18.25      ,
          0.74001724],
       [ 59.        ,  59.        ,  69.        ,  56.        ,
         39.        ,  36.75  

6